In [60]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
from collections import deque
import pandas as pd
from pandas import Series
import numpy as np
import requests

In [333]:
df = pd.read_csv('donation.csv', encoding='utf-8')

In [334]:
df.head()

,Unnamed: 0,categories,title,company_names,percentages,funding_amounts,target_amounts,n_supporters,likes,descriptions,durations,days_left
0,0,기부·후원,전자오락 수호대를 모집합니다! - 서촌에서 가장 오래된 오락실 되살리기,설재우,117%,"4,706,000원","4,000,000원",40,4,청소년들의 드림센터 용오락실 복원하기,2015.05.06-2015.05.20,None
1,1,기부·후원,청소년 백패킹! “나를 찾는 여행”을 준비합니다!,Choi Yong Mo,0%,"30,000원","6,256,000원",3,0,배낭속에 자신의 삶과 꿈을 담아 나르는 여행,2015.05.12-2015.09.09,펀딩성공
2,2,기부·후원,흐로닝언 유학생 박현후인턴 한국음식 보내기 프로젝트,김희정,144%,"369,282원","255,000원",22,7,NaN,2016.02.05-2016.02.25,None
3,3,기부·후원,[네팔 긴급구호] 한순간 무너진 네팔 카트만두,휴면계정,0%,0원,"3,000,000원",0,0,[네팔 긴급구호] 한순간 무너진 네팔 카트만두,2015.04.27-2015.05.11,펀딩성공
4,4,기부·후원,사이클선수 민혜가 다시 일어설 수 있도록 희망을 선물해 주세요.,구성은(사이클 선수),160%,"8,028,330원","5,000,000원",266,52,자전거로 인연을 맺어 오랜시간 함께해 온 동료이자 한국사이클의 여제였던 이민혜 선수...,2018.07.31-2018.09.02,None


In [335]:
# 형변환
df['percentages'] = df['percentages'].str.replace('%','').astype('int64')
df['funding_amounts'] = df['funding_amounts'].str.replace(',','')
df['funding_amounts'] = df['funding_amounts'].str.replace('원','').astype('int64')
df['target_amounts'] = df['target_amounts'].str.replace(',','')
df['target_amounts'] = df['target_amounts'].str.replace('원','').astype('int64')
df['n_supporters'] = df['n_supporters'].astype('int64')
df['likes'] = df['likes'].astype('int64')

In [336]:
# durations split
df['durations'] = df['durations'].str.split('-')

In [337]:
# durations split 진행 후 start,end 특성 추가
df['start'] = df['durations'].str.get(0)
df['end'] = df['durations'].str.get(1)

In [338]:
# start, end 형변환(object → datetime)
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

In [339]:
# 진행기간(time) 계산 (start - end)
df['time'] = -(df['start'] - df['end'])

In [340]:
# 진행기간(time) 형변환 (timedelta64 → int64)
df['time'] = df['time'].dt.days

In [341]:
df.head()

,Unnamed: 0,categories,title,company_names,percentages,funding_amounts,target_amounts,n_supporters,likes,descriptions,durations,days_left,start,end,time
0,0,기부·후원,전자오락 수호대를 모집합니다! - 서촌에서 가장 오래된 오락실 되살리기,설재우,117,4706000,4000000,40,4,청소년들의 드림센터 용오락실 복원하기,"[2015.05.06, 2015.05.20]",None,2015-05-06,2015-05-20,14
1,1,기부·후원,청소년 백패킹! “나를 찾는 여행”을 준비합니다!,Choi Yong Mo,0,30000,6256000,3,0,배낭속에 자신의 삶과 꿈을 담아 나르는 여행,"[2015.05.12, 2015.09.09]",펀딩성공,2015-05-12,2015-09-09,120
2,2,기부·후원,흐로닝언 유학생 박현후인턴 한국음식 보내기 프로젝트,김희정,144,369282,255000,22,7,NaN,"[2016.02.05, 2016.02.25]",None,2016-02-05,2016-02-25,20
3,3,기부·후원,[네팔 긴급구호] 한순간 무너진 네팔 카트만두,휴면계정,0,0,3000000,0,0,[네팔 긴급구호] 한순간 무너진 네팔 카트만두,"[2015.04.27, 2015.05.11]",펀딩성공,2015-04-27,2015-05-11,14
4,4,기부·후원,사이클선수 민혜가 다시 일어설 수 있도록 희망을 선물해 주세요.,구성은(사이클 선수),160,8028330,5000000,266,52,자전거로 인연을 맺어 오랜시간 함께해 온 동료이자 한국사이클의 여제였던 이민혜 선수...,"[2018.07.31, 2018.09.02]",None,2018-07-31,2018-09-02,33
